<a href="https://colab.research.google.com/github/stesalps/pmcp/blob/main/Copy_of_colab_models_as_a_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab AI: List available models

In [ ]:
from google.colab import ai
ai.list_models()

['google/gemini-2.0-flash',
 'google/gemini-2.0-flash-lite',
 'google/gemini-2.5-flash',
 'google/gemini-2.5-flash-lite',
 'google/gemini-2.5-pro',
 'google/gemma-3-12b',
 'google/gemma-3-1b',
 'google/gemma-3-27b',
 'google/gemma-3-4b']

The model names give you a hint about their capabilities and intended use:

Pro: These are the most capable models, ideal for complex reasoning, creative tasks, and detailed analysis.

Flash: These models are optimized for high speed and efficiency, making them great for summarization, chat applications, and tasks requiring rapid responses.

Gemma: These are lightweight, open-weight models suitable for a variety of text generation tasks and are great for experimentation.

# Colab AI: Choose a different model

In [ ]:
from google.colab import ai

response = ai.generate_text("What is the capital of England", model_name='google/gemini-2.0-flash-lite')
print(response)

The capital of England is **London**.



# Colab AI: Simple batch generation example

In [ ]:
# Only text-to-text input/output is supported
from google.colab import ai

response = ai.generate_text("What is the capital of France?")
print(response)

# Colab AI: Simple streaming example

In [ ]:
from google.colab import ai

stream = ai.generate_text("Tell me a short story.", stream=True)
for text in stream:
  print(text, end='')

# Colab AI: Formatted streaming example

In [ ]:
#code is not necessary for colab.ai, but is useful in fomatting text chunks
import sys
from google.colab import ai


class LineWrapper:
    def __init__(self, max_length=80):
        self.max_length = max_length
        self.current_line_length = 0

    def print(self, text_chunk):
        i = 0
        n = len(text_chunk)
        while i < n:
            start_index = i
            while i < n and text_chunk[i] not in ' \n': # Find end of word
                i += 1
            current_word = text_chunk[start_index:i]

            delimiter = ""
            if i < n: # If not end of chunk, we found a delimiter
                delimiter = text_chunk[i]
                i += 1 # Consume delimiter

            if current_word:
                needs_leading_space = (self.current_line_length > 0)

                # Case 1: Word itself is too long for a line (must be broken)
                if len(current_word) > self.max_length:
                    if needs_leading_space: # Newline if current line has content
                        sys.stdout.write('\n')
                        self.current_line_length = 0
                    for char_val in current_word: # Break the long word
                        if self.current_line_length >= self.max_length:
                            sys.stdout.write('\n')
                            self.current_line_length = 0
                        sys.stdout.write(char_val)
                        self.current_line_length += 1
                # Case 2: Word doesn't fit on current line (print on new line)
                elif self.current_line_length + (1 if needs_leading_space else 0) + len(current_word) > self.max_length:
                    sys.stdout.write('\n')
                    sys.stdout.write(current_word)
                    self.current_line_length = len(current_word)
                # Case 3: Word fits on current line
                else:
                    if needs_leading_space:
                        # Define punctuation that should not have a leading space
                        # when they form an entire "word" (token) following another word.
                        no_leading_space_punctuation = {
                            ",", ".", ";", ":", "!", "?",        # Standard sentence punctuation
                            ")", "]", "}",                     # Closing brackets
                            "'s", "'S", "'re", "'RE", "'ve", "'VE", # Common contractions
                            "'m", "'M", "'ll", "'LL", "'d", "'D",
                            "n't", "N'T",
                            "...", "…"                          # Ellipses
                        }
                        if current_word not in no_leading_space_punctuation:
                            sys.stdout.write(' ')
                            self.current_line_length += 1
                    sys.stdout.write(current_word)
                    self.current_line_length += len(current_word)

            if delimiter == '\n':
                sys.stdout.write('\n')
                self.current_line_length = 0
            elif delimiter == ' ':
                # If line is full and a space delimiter arrives, it implies a wrap.
                if self.current_line_length >= self.max_length:
                    sys.stdout.write('\n')
                    self.current_line_length = 0

        sys.stdout.flush()


wrapper = LineWrapper()
for chunk in ai.generate_text('Give me a long winded description about the evolution of the Roman Empire.', model_name='google/gemini-2.0-flash', stream=True):
  wrapper.print(chunk)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Server Launch and GUI - v7.0.x (Updated for Ollama-powered Box 2 with Chat & Agent Selector) ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - FastAPI server launch with Uvicorn (Integrates Box 2 or runs proxy)                                   ║
# ║ - Multi-interface support: Jupyter, Gradio                                                          ║
# ║ - Plugin manifests for AI integration (Claude, OpenAI)                                                  ║
# ║ - System integration and monitoring                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 3: Initializing Server Launch and GUI Systems (Updated for Ollama-powered Box 2 with Chat & Agent Selector)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading configurations from previous boxes (Updated Paths)...")

# --- Load Box 1 and Box 2 configurations ---
try:
    # Standardized config directory path from updated Box 1
    config_dir = Path("/content/drive/MyDrive/UnifiedManusSystem/config")
    # Fallback for local runs
    if not config_dir.exists():
        config_dir = Path("./UnifiedManusSystem/config")

    if not config_dir.exists():
        raise FileNotFoundError("Config directory not found")

    # Load Box 1 config
    box1_config_file = config_dir / "box1_exports.json"
    if box1_config_file.exists():
        with open(box1_config_file, "r") as f:
            box1_config = json.load(f)
        print("✅ Box 1 configuration loaded")
    else:
        raise FileNotFoundError("Box 1 config not found")

    # Load Box 2 config (now includes Ollama details)
    box2_config_file = config_dir / "box2_exports.json"
    if box2_config_file.exists():
        with open(box2_config_file, "r") as f:
            box2_config = json.load(f)
        print("✅ Box 2 configuration loaded")
    else:
        print("⚠️ Box 2 config not found, will use defaults/fallbacks")
        box2_config = {"tools_registered": [], "agent_session": "unknown", "api_endpoints": [], "ollama_model": "unknown"}

    # Extract configuration
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]

    tools_available = box2_config.get("tools_registered", [])
    agent_session = box2_config.get("agent_session", "unknown")
    box2_api_endpoints = box2_config.get("api_endpoints", [])
    ollama_model = box2_config.get("ollama_model", "unknown")

    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"🤖 Agent Session: {agent_session}")
    print(f"🦙 Ollama Model: {ollama_model}")
    print(f"🛠️ Tools Available: {len(tools_available)}")

except Exception as e:
    print(f"❌ Error loading configurations: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True
    tools_available = []
    agent_session = "unknown"
    box2_api_endpoints = []
    ollama_model = "unknown"

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules for Box 3...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core web framework
import uvicorn
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
import requests # For proxying calls to Box 2 if needed

# GUI frameworks
GRADIO_AVAILABLE = False
JUPYTER_AVAILABLE = False
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")

try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

print("✅ All Box 3 modules imported successfully")

print("🔄 Step 3: Re-establishing connection to Box 2 components...")

# --- Determine if Box 2 is running ---
# Check if Box 2's app and agent are available in the current session (e.g., if this is a unified script)
box2_running_in_session = 'app' in globals() and 'agent' in globals() and 'TOOL_REGISTRY' in globals()
box2_api_accessible = False
box2_api_url = "http://localhost:8000" # Default assumption for internal calls

if box2_running_in_session:
    print("✅ Box 2 components found in current session (unified script mode)")
    box2_running = True
    # Use the in-session components
    try:
        from __main__ import app as box2_app, agent as box2_agent, TOOL_REGISTRY as box2_tool_registry
        print("🔗 Linked to in-session Box 2 components")
    except ImportError:
        print("⚠️ Could not import Box 2 components directly, using global references if available")
        # They are already in globals if the check passed
        box2_app = globals().get('app')
        box2_agent = globals().get('agent')
        box2_tool_registry = globals().get('TOOL_REGISTRY')
else:
    # Check if Box 2 server is running externally
    try:
        response = requests.get(f"{box2_api_url}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Box 2 server is accessible externally")
            box2_running = True
            box2_api_accessible = True
        else:
            print(f"⚠️ Box 2 server health check failed (Status: {response.status_code})")
            box2_running = False
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not connect to Box 2 server at {box2_api_url}: {e}")
        box2_running = False

if not box2_running:
    print("⚠️ Box 2 components not found/runnable. GUI will use fallback methods or fail gracefully.")


print("📄 Step 4: Creating plugin manifest files...")

def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)
    static_dir = site_dir / "static"
    static_dir.mkdir(exist_ok=True)

    # AI Plugin manifest (OpenAI/Claude compatible structure)
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support, powered by Ollama LLM.",
        "description_for_model": f"A unified agent system with file operations, Python execution, package management, and multi-role thinking capabilities via the 'action_agent' tool, using the {ollama_model} model. Also supports direct chat via /mcp/chat.",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"}, # Points to Box 2's OpenAPI spec
        "logo_url": f"{public_url}/site/static/logo.png", # Placeholder
        "contact_email": "support@example.com",
        "legal_info_url": f"{public_url}/site/legal.html" # Placeholder
    }
    try:
        with open(site_dir / "ai-plugin.json", "w") as f:
            json.dump(ai_plugin_manifest, f, indent=2)
        print("✅ AI Plugin manifest created")
    except Exception as e:
        print(f"❌ Failed to create AI Plugin manifest: {e}")

    # Claude-compatible manifest (YAML)
    claude_manifest = {
        "name": "unified_manus",
        "description": f"Multi-agent coding assistant with comprehensive tool support, including an 'action_agent' for complex tasks and a '/mcp/chat' endpoint for direct conversation, powered by Ollama ({ollama_model}).",
        "version": "7.0.x",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            "chat": f"{public_url}/mcp/chat" # Include the new chat endpoint
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management", "chat"]
    }
    try:
        import yaml # Should be available as installed by updated Box 1
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
        print("✅ Claude manifest (YAML) created")
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")
    except Exception as e:
        print(f"❌ Failed to create Claude manifest: {e}")

    # Simple index.html for /site/
    index_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Unified Manus System (Ollama)</title>
</head>
<body>
    <h1>🤖 Unified Manus MCP System v7.0.x (Ollama-powered)</h1>
    <p>Multi-Agent Coding Assistant</p>
    <ul>
        <li><a href="/docs">API Documentation (FastAPI)</a></li>
        <li><a href="/redoc">API Documentation (ReDoc)</a></li>
        <li>Agent Session: {agent_session}</li>
        <li>Ollama Model: {ollama_model}</li>
        <li>Tools Available: {len(tools_available)}</li>
        <li>Public URL: <a href="{public_url}">{public_url}</a></li>
    </ul>
</body>
</html>
"""
    try:
        with open(site_dir / "index.html", "w") as f:
            f.write(index_content)
        print("✅ Basic site index.html created")
    except Exception as e:
        print(f"❌ Failed to create site index.html: {e}")

create_plugin_manifests()


print("🎨 Step 5: Setting up Gradio and Jupyter interfaces...")

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result_dict = box2_agent.run_task(task, context)
                if result_dict.get("status") == "success":
                     return result_dict.get("final_output", "No final output found.")
                else:
                     return f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}"
            elif box2_api_accessible:
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                 if response.status_code == 200:
                     result_dict = response.json()
                     if result_dict.get("status") == "success":
                          return result_dict.get("final_output", "No final output found in API response.")
                     else:
                          return f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}"
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 from __main__ import TOOL_REGISTRY
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                from __main__ import TOOL_REGISTRY
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    # --- New Chat Functionality for Gradio ---
    def chat_with_model(message_history):
        """
        Chat with the model via the new /mcp/chat endpoint.
        Gradio's ChatInterface passes message_history as a list of [user_msg, bot_msg, user_msg, ...]
        We need to convert it to the format expected by /mcp/chat: [{"role": "...", "content": "..."}]
        """
        # Convert Gradio history to Ollama format
        ollama_messages = []
        for i, msg in enumerate(message_history):
            if i % 2 == 0: # User message
                ollama_messages.append({"role": "user", "content": msg})
            else: # Bot message
                ollama_messages.append({"role": "assistant", "content": msg})

        if box2_running_in_session and box2_api_accessible:
            # Prefer direct API call for streaming
            try:
                payload = {"messages": ollama_messages}
                # Note: Streaming responses from external APIs to Gradio chatbot can be complex.
                # A simpler approach is to make a non-streaming call.
                response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120)
                if response.status_code == 200:
                    # The /mcp/chat endpoint returns a stream. If we get a non-stream response here,
                    # it might be an aggregated final message or an error.
                    # Let's try to parse JSON. If it fails, return raw text.
                    try:
                        data = response.json()
                        # Check for common fields in the final aggregated response
                        if "message" in data:
                            # If it's the final message structure from the streaming endpoint
                            return data.get("message", {}).get("content", "Received message, content unclear (format 1).")
                        elif "content" in data:
                             # If it's a simple content response
                             return data["content"]
                        elif "final_output" in data: # Maybe the action_agent format was used somehow
                            return data["final_output"]
                        else:
                            # Return the whole JSON if structure is unknown
                            return f"Received JSON, structure unclear: {data}"
                    except json.JSONDecodeError:
                        # If response isn't JSON, return text content
                        return response.text or "Received response, but it was empty."
                else:
                    return f"❌ Chat API Error ({response.status_code}): {response.text}"
            except Exception as e:
                 return f"❌ Chat API Call Failed: {str(e)}"
        elif box2_running_in_session:
            # If only running in session, we'd need a way to call the Ollama chat function directly
            # and handle streaming. This is more complex in Gradio without async support in this context.
            # Fallback: Use a simple non-streaming direct call (if such a function exists or is adapted).
            # For now, indicate it's not fully supported via direct call in this GUI setup.
            return "⚠️ Direct chat not fully implemented in this GUI mode. Use API or Jupyter GUI for full chat."
        else:
             return "❌ Box 2 Chat API is not accessible."

    with gr.Blocks(title="Unified Manus MCP System (Ollama)", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x (Ollama-powered)")
        gr.Markdown("Multi-Agent Coding Assistant with LLM Capabilities")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=20, max_lines=30, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=15, max_lines=20, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("Direct Chat (via /mcp/chat)"):
             # Gradio's ChatInterface is a convenient way to handle chat
             chatbot = gr.Chatbot(label="Conversation")
             msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
             clear_chat = gr.Button("Clear Chat")

             def respond(message, chat_history):
                 # Append user message to history
                 chat_history.append((message, None))
                 # Get response from chat function
                 bot_message = chat_with_model([item for sublist in chat_history for item in sublist if item is not None])
                 # Update the last entry in history with the bot's response
                 chat_history[-1] = (message, bot_message)
                 return "", chat_history

             msg.submit(respond, [msg, chatbot], [msg, chatbot])
             clear_chat.click(fn=lambda: ([], []), inputs=[], outputs=[chatbot, msg], queue=False)


        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Ollama Model: {ollama_model}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# --- Jupyter Interface ---
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")

    # --- Jupyter UI Logic ---
    def create_jupyter_ui():
        clear_output(wait=True)

        # --- UI Elements ---
        task_input = widgets.Text(
            value='',
            placeholder='Enter a task for the agent (e.g., Write a Python script to calculate Fibonacci numbers)',
            description='Task:',
            disabled=False,
            layout=widgets.Layout(width='100%')
        )

        context_input = widgets.Textarea(
            value='',
            placeholder='Optional context for the task',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        run_button = widgets.Button(
            description="Run Task with Action Agent",
            button_style='success',
            tooltip='Execute the task using the internal Ollama-powered ManusAgent',
            icon='play'
        )

        tool_name_dropdown = widgets.Dropdown(
            options=tools_available if tools_available else ['No tools loaded'],
            value=tools_available[0] if tools_available else 'No tools loaded',
            description='Tool:',
            disabled=not tools_available,
        )

        tool_input_textarea = widgets.Textarea(
            value='{}',
            placeholder='Enter tool input as JSON (e.g., {"file_path": "test.txt", "content": "Hello"})',
            description='Input (JSON):',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        tool_run_button = widgets.Button(
            description="Execute Tool",
            button_style='info',
            tooltip='Run the selected tool with the provided input',
            icon='cogs'
        )

        clear_button = widgets.Button(
            description="Clear Output",
            button_style='warning',
            tooltip='Clear the output areas below',
            icon='eraser'
        )

        output_area = widgets.Output(
            layout=widgets.Layout(height='400px', border='1px solid black', overflow='auto', padding='10px')
        )

        # --- Chat Elements ---
        chat_history_area = widgets.Output(
             layout=widgets.Layout(height='300px', border='1px solid blue', overflow='auto', padding='10px', background_color='#f0f8ff')
        )
        chat_input = widgets.Textarea(
            value='',
            placeholder='Type your message here for direct chat...',
            description='Chat:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='80px')
        )
        # --- Agent Selection Dropdown for Chat ---
        chat_agent_selector = widgets.Dropdown(
            options=[
                ("Direct Chat with Ollama Model", "direct_chat"),
                ("Chat via Action Agent Team", "action_agent")
            ],
            value="action_agent", # Default to action agent
            description="Chat Agent:",
            disabled=False,
        )
        chat_send_button = widgets.Button(
            description="Send Message",
            button_style='primary',
            tooltip='Send message to the selected agent',
            icon='paper-plane'
        )
        chat_clear_button = widgets.Button(
            description="Clear Chat",
            button_style='warning',
            tooltip='Clear the chat history',
            icon='trash'
        )

        # --- Event Handlers ---
        def on_run_task(b):
            task = task_input.value
            context = context_input.value
            if not task:
                with output_area:
                    print("⚠️ Please enter a task.")
                return

            with output_area:
                clear_output(wait=True)
                print(f"🚀 Running task: '{task}'")
                if context:
                    print(f"   Context: '{context}'")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        result_dict = box2_agent.run_task(task, context)
                        if result_dict.get("status") == "success":
                             print("✅ Task completed!")
                             print("\n📝 Plan:")
                             print(result_dict.get("plan", "N/A"))
                             print("\n💻 Generated Code:")
                             print(result_dict.get("code", "N/A"))
                             print("\n🔍 Review:")
                             print(result_dict.get("review", "N/A"))
                             print("\n--- Final Output ---")
                             print(result_dict.get("final_output", "N/A"))
                        else:
                             print(f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}")
                    elif box2_api_accessible:
                         payload = {"task": task, "context": context}
                         response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                         if response.status_code == 200:
                             result_dict = response.json()
                             if result_dict.get("status") == "success":
                                  print("✅ Task completed!")
                                  print("\n📝 Plan:")
                                  print(result_dict.get("plan", "N/A"))
                                  print("\n💻 Generated Code:")
                                  print(result_dict.get("code", "N/A"))
                                  print("\n🔍 Review:")
                                  print(result_dict.get("review", "N/A"))
                                  print("\n--- Final Output ---")
                                  print(result_dict.get("final_output", "N/A"))
                             else:
                                  print(f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}")
                         else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Agent Core) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR running task: {e}")
                    traceback.print_exc()

        def on_execute_tool(b):
            tool_name = tool_name_dropdown.value
            tool_input_str = tool_input_textarea.value

            if tool_name == 'No tools loaded':
                 with output_area:
                     print("⚠️ No tools are available to execute.")
                 return

            try:
                if tool_input_str.strip():
                    tool_input_data = json.loads(tool_input_str)
                else:
                    tool_input_data = {}
            except json.JSONDecodeError as e:
                 with output_area:
                     print(f"❌ Invalid JSON input for tool: {e}")
                 return

            with output_area:
                clear_output(wait=True)
                print(f"🔧 Executing tool: '{tool_name}'")
                print(f"   Input: {json.dumps(tool_input_data, indent=2)}")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        if tool_name in box2_tool_registry:
                            result = box2_tool_registry[tool_name](**tool_input_data)
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result, indent=2) if isinstance(result, dict) else str(result))
                        else:
                            print(f"❌ Tool '{tool_name}' not found in registry.")
                    elif box2_api_accessible:
                        payload = {"tool_name": tool_name, "tool_input": tool_input_data}
                        response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                        if response.status_code == 200:
                            res_data = response.json()
                            result_content = res_data.get("result", "No result field")
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result_content, indent=2) if isinstance(result_content, dict) else str(result_content))
                        else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Tool Execution) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR executing tool: {e}")
                    traceback.print_exc()

        def on_clear(b):
            output_area.clear_output()

        # --- Chat Event Handlers (Revised) ---
        def on_chat_send(b):
            user_message = chat_input.value.strip()
            selected_agent = chat_agent_selector.value
            if not user_message:
                with chat_history_area:
                    print("⚠️ Please enter a message to send.")
                return

            with chat_history_area:
                print(f"[You]: {user_message}")

            try:
                if selected_agent == "direct_chat" and box2_api_accessible:
                    # Call the new /mcp/chat endpoint for direct conversation
                    messages = [{"role": "user", "content": user_message}]
                    payload = {"messages": messages}
                    response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120, stream=True)
                    if response.status_code == 200:
                        full_response = ""
                        with chat_history_area:
                            print(f"[Assistant ({ollama_model})]: ", end="", flush=True)
                        for line in response.iter_lines():
                            if line:
                                try:
                                    chunk_data = json.loads(line)
                                    if chunk_data.get("event") == "message":
                                        content = chunk_data.get("data", {}).get("content", "")
                                        full_response += content
                                        with chat_history_area:
                                            print(content, end="", flush=True)
                                    elif chunk_data.get("event") == "end":
                                        break
                                    elif chunk_data.get("event") == "error":
                                        error_msg = chunk_data.get("data", {}).get("error", "Unknown stream error")
                                        with chat_history_area:
                                            print(f"\n❌ Stream Error: {error_msg}")
                                        break
                                except json.JSONDecodeError:
                                    with chat_history_area:
                                        print(f"\n⚠️ Malformed stream data received.")
                        with chat_history_area:
                             print() # Newline after streaming
                        chat_input.value = ''
                    else:
                         with chat_history_area:
                             print(f"\n❌ Chat API Error ({response.status_code}): {response.text}")
                         chat_input.value = ''

                elif selected_agent == "action_agent":
                    # Send message to the Action Agent Team
                    # Attempt to capture context from output_area
                    # Note: Directly reading from widgets.Output is tricky.
                    # We pass a generic context string. A more advanced version
                    # might store the last output in a variable.
                    context_from_output = "See previous output in the 'Output' section above for context."

                    with chat_history_area:
                        print(f"[Assistant (Action Agent Team)]: Thinking... ", end="", flush=True)

                    try:
                        if box2_running_in_session:
                            # Direct call to in-session agent
                            result_dict = box2_agent.run_task(goal=user_message, context=context_from_output)
                            final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary.]')
                        elif box2_api_accessible:
                            # Call Box 2 API
                            payload = {"task": user_message, "context": context_from_output}
                            api_response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                            if api_response.status_code == 200:
                                result_dict = api_response.json()
                                final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary via API.]')
                            else:
                                final_reply = f"❌ API Error contacting Action Agent: {api_response.status_code} - {api_response.text}"
                        else:
                            final_reply = "❌ Neither direct session nor API access to Box 2 is available for the Action Agent."

                        with chat_history_area:
                            print() # Newline after "Thinking..."
                            print(f"[Assistant (Action Agent Team)]: {final_reply}")
                            print("-" * 20)

                    except Exception as e:
                         with chat_history_area:
                             print() # Newline after "Thinking..."
                             print(f"[System Error (Action Agent)]: {str(e)}")

                    chat_input.value = '' # Clear input after action agent call

                else:
                    # Fallback if direct chat is selected but API is not accessible
                    with chat_history_area:
                        print(f"\n❌ Selected agent '{selected_agent}' is not available in the current mode.")
                    chat_input.value = ''

            except Exception as e:
                 with chat_history_area:
                     print(f"\n💥 ERROR sending chat message: {e}")
                 chat_input.value = '' # Clear input on general error


        def on_chat_clear(b):
            chat_history_area.clear_output()
            chat_input.value = '' # Also clear the input field

        # Assign event handlers
        run_button.on_click(on_run_task)
        tool_run_button.on_click(on_execute_tool)
        clear_button.on_click(on_clear)
        chat_send_button.on_click(on_chat_send)
        chat_clear_button.on_click(on_chat_clear)

        # --- Display Layout ---
        ui_layout = widgets.VBox([
            widgets.HTML("<h1>🤖 Unified Manus MCP System (Jupyter - Ollama)</h1>"),
            widgets.HTML("<h2>Agent Task Execution</h2>"),
            task_input,
            context_input,
            run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Tool Execution</h2>"),
            tool_name_dropdown,
            tool_input_textarea,
            tool_run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Direct Chat</h2>"),
            chat_agent_selector, # Add the agent selector dropdown
            chat_history_area,
            chat_input,
            widgets.HBox([chat_send_button, chat_clear_button]),
            widgets.HTML("<h2 style='margin-top: 20px;'>Output</h2>"),
            clear_button,
            output_area
        ])

        display(ui_layout)

    return create_jupyter_ui


print("🌐 Step 6: Setting up FastAPI server (Integrated or Proxy)...")

# --- Integrated or Proxy FastAPI App for Box 3 ---
def create_integrated_or_proxy_server():
    """Create the FastAPI app for Box 3, either integrated or proxying to Box 2"""
    app = FastAPI(
        title="Unified Manus MCP Server - Box 3 (Ollama)",
        description="Integrated server with GUI interfaces and potential proxying to Ollama-powered Box 2",
        version="7.0.x"
    )

    # CORS middleware
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"], # Adjust for production
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    # Static files
    site_dir = BASE_DIR / "site"
    if site_dir.exists():
        app.mount("/site", StaticFiles(directory=str(site_dir)), name="site")

    # --- Root endpoint ---
    @app.get("/")
    async def root():
        """Root endpoint"""
        index_file = site_dir / "index.html"
        if index_file.exists():
            return FileResponse(str(index_file))
        else:
            return {
                "message": "Unified Manus MCP System - Box 3 (Ollama-powered)",
                "version": "7.0.x",
                "box2_status": "Integrated" if box2_running_in_session else ("Proxying" if box2_api_accessible else "Unavailable"),
                "public_url": public_url,
                "agent_session": agent_session,
                "ollama_model": ollama_model
            }

    # --- Health check ---
    @app.get("/health")
    async def health():
        """Health check endpoint"""
        box2_status = "Unavailable"
        if box2_running_in_session:
            box2_status = "Integrated"
        elif box2_api_accessible:
            try:
                b2_response = requests.get(f"{box2_api_url}/health", timeout=2)
                if b2_response.status_code == 200:
                    b2_data = b2_response.json()
                    if b2_data.get("status") == "healthy":
                        box2_status = "Accessible (Healthy)"
                    else:
                        box2_status = f"Accessible (Degraded: {b2_data.get('status')})"
                else:
                    box2_status = f"Accessible (API Error: {b2_response.status_code})"
            except:
                box2_status = "Accessible (Health Check Failed)"

        return {
            "status": "healthy" if ("Healthy" in box2_status or box2_status == "Integrated") else "degraded",
            "box": 3,
            "version": "7.0.x",
            "timestamp": datetime.now().isoformat(),
            "box2_status": box2_status,
            "agent_session": agent_session,
            "ollama_model": ollama_model
        }

    # --- Proxy endpoints to Box 2 if it's running externally ---
    if not box2_running_in_session and box2_api_accessible:
        print("🔄 Setting up proxy endpoints to external Box 2...")

        @app.api_route("/mcp/tools/call", methods=["POST"])
        async def proxy_tool_call(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/tools/call", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/tools/list", methods=["GET"])
        async def proxy_tool_list(request: Request):
             try:
                 params = dict(request.query_params)
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", params=params)
                 return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
             except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/action", methods=["POST"])
        async def proxy_agent_action(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/agent/action", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/memory", methods=["GET", "POST"])
        async def proxy_agent_memory(request: Request):
            try:
                url = f"{box2_api_url}/mcp/agent/memory"
                if request.method == "POST":
                    url += "/clear"
                body = await request.body() if request.method in ["POST", "PUT"] else None
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                if body:
                    response = requests.request(request.method, url, data=body, headers=headers)
                else:
                     response = requests.request(request.method, url, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        # --- Proxy the new Chat endpoint ---
        @app.api_route("/mcp/chat", methods=["POST"])
        async def proxy_chat(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                # Use requests to forward the stream
                # Note: True proxying of SSE streams is complex with standard requests.
                # A more robust solution might use `httpx` or async libraries.
                # For now, we proxy the request and return the response.
                response = requests.post(f"{box2_api_url}/mcp/chat", data=body, headers=headers, stream=True)
                # Returning a streaming response correctly requires more setup.
                # This is a simplified proxy that might not handle streams perfectly.
                # Consider using `httpx` StreamResponse for better SSE proxying.
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Chat Proxy error: {str(e)}"})


    elif box2_running_in_session:
        print("🔗 Box 2 is integrated, no proxy needed for its endpoints.")
    else:
        print("⚠️ Box 2 is not accessible, proxy endpoints will not function.")

    return app

# Create the integrated/proxy app
integrated_app = create_integrated_or_proxy_server()
print("✅ FastAPI server (Box 3) configured")


print("💾 Step 7: Saving Box 3 configuration...")

def save_box3_state():
    """Save Box 3 state"""
    state = {
        "interfaces_available": ["jupyter", "gradio", "api"],
        "web_interface_ready": True,
        "plugin_manifests_created": True,
        "launch_modes": ["jupyter", "gradio", "api", "all"],
        "timestamp": datetime.now().isoformat(),
        "box2_connection": {
            "in_session": box2_running_in_session,
            "api_accessible": box2_api_accessible,
            "status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable")
        },
        "gradio_available": GRADIO_AVAILABLE,
        "jupyter_available": JUPYTER_AVAILABLE,
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "agent_session": agent_session,
        "ollama_model": ollama_model
    }
    config_file = BASE_DIR / "config" / "box3_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 3 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 3 state: {e}")

save_box3_state()

print("🔍 Step 8: Final verification...")

def verify_box3_setup():
    """Verify Box 3 setup"""
    checks = {
        "Web Interface": (BASE_DIR / "site" / "index.html").exists(),
        "Plugin Manifests": (BASE_DIR / "site" / "ai-plugin.json").exists(),
        "Configuration Files": (BASE_DIR / "config" / "box3_exports.json").exists(),
        "Site Directory": (BASE_DIR / "site").exists(),
        "Box 1 Config": (BASE_DIR / "config" / "box1_exports.json").exists(),
        "Box 2 Config": (BASE_DIR / "config" / "box2_exports.json").exists(),
        "Box 2 Connection": box2_running_in_session or box2_api_accessible,
        "Gradio Availability": not GRADIO_AVAILABLE or (GRADIO_AVAILABLE and setup_gradio_interface() is not None),
        "Jupyter Availability": not JUPYTER_AVAILABLE or (JUPYTER_AVAILABLE and setup_jupyter_interface() is not None)
    }
    print("🔍 Box 3 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box3_setup()

if verification_passed:
    print("🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("✅ ALL SYSTEMS VERIFIED AND READY!")
    print("🚀 LAUNCH OPTIONS:")
    print(" • Jupyter Interface: launch_system('jupyter')")
    print(" • Gradio Interface: launch_system('gradio')")
    print(" • API Server Only: launch_system('api')")
    print(" • ALL Interfaces: launch_system('all')")
    print(f"🌍 URLs:")
    print(f" • Main API: {public_url}")
    print(f" • Dashboard: {dashboard_url}")
    print(f" • Web Interface: {public_url}/site/")
    print(f" • API Docs: {public_url}/docs")
    print(f"📁 System Directories:")
    print(f" • Base: {BASE_DIR}")
    print(f" • Workspace: {WORKSPACE_DIR}")
    print(f" • Logs: {BASE_DIR / 'logs'}")
    print(f" • Site: {BASE_DIR / 'site'}")
    print(f"🔧 System Status:")
    print(f" • Environment: {'Google Colab' if IS_COLAB else 'Local'}")
    print(f" • Box 2 Status: {'Integrated' if box2_running_in_session else ('Proxying' if box2_api_accessible else 'Unavailable')}")
    print(f" • Ollama Model: {ollama_model}")
    print(f" • Gradio Ready: {'Yes' if GRADIO_AVAILABLE else 'No'}")
    print(f" • Jupyter Ready: {'Yes' if JUPYTER_AVAILABLE else 'No'}")
    print("=" * 80)
    print("🔄 Box 3 is ready for launch!")
else:
    print("❌ BOX 3 SETUP HAD ISSUES!")
    print("Please check the errors above.")

# --- Launch Functions ---
def launch_system(mode: str = "jupyter"):
    """
    Launch the system in different modes.
    Modes: 'jupyter', 'gradio', 'api', 'all'
    """
    global integrated_app # Use the app created earlier

    if mode == "jupyter":
        if JUPYTER_AVAILABLE:
            print("📓 Launching Jupyter interface...")
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_ui()
            else:
                print("❌ Failed to setup Jupyter interface.")
        else:
            print("❌ Jupyter is not available in this environment.")

    elif mode == "gradio":
        if GRADIO_AVAILABLE:
            print("🎨 Launching Gradio interface...")
            demo = setup_gradio_interface()
            if demo:
                demo.launch(share=True, server_name="0.0.0.0", server_port=7860)
                print(f"✅ Gradio launched. Access at: http://localhost:7860 (or the public Gradio link if shared)")
            else:
                print("❌ Failed to setup Gradio interface.")
        else:
            print("❌ Gradio is not available.")

    elif mode == "api":
        print("🌐 Launching FastAPI server (Box 3)...")
        uvicorn.run(integrated_app, host="0.0.0.0", port=8000)
        print(f"✅ FastAPI server launched. Access at: {public_url}")

    elif mode == "all":
        print("🔄 Launching all interfaces...")
        def run_api():
             uvicorn.run(integrated_app, host="0.0.0.0", port=8000)

        api_thread = threading.Thread(target=run_api)
        api_thread.daemon = True
        api_thread.start()
        print(f"✅ API Server started in background on port 8000")

        time.sleep(2)

        if JUPYTER_AVAILABLE:
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_thread = threading.Thread(target=jupyter_ui)
                jupyter_thread.start()
                print("✅ Jupyter interface launched")
            else:
                 print("⚠️ Jupyter interface setup failed")

        if GRADIO_AVAILABLE:
            demo = setup_gradio_interface()
            if demo:
                def run_gradio():
                    demo.launch(share=True, server_name="0.0.0.0", server_port=7860, prevent_thread_lock=True)
                    demo.block_thread()

                gradio_thread = threading.Thread(target=run_gradio)
                gradio_thread.daemon = True
                gradio_thread.start()
                print("✅ Gradio interface launched")
                time.sleep(3)
            else:
                 print("⚠️ Gradio interface setup failed")

        print("🎉 All requested interfaces started!")
        print(f"🌍 API: {public_url}")
        print(f"📊 Dashboard: {dashboard_url} (if applicable)")
        if GRADIO_AVAILABLE:
            print(f"🎨 Gradio: Check output above or http://localhost:7860")
        print("ℹ️ Main thread will now idle. Stop with KeyboardInterrupt (Ctrl+C).")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Shutting down all services...")

    else:
        print(f"❌ Unknown mode: {mode}")
        print("Available modes: jupyter, gradio, api, all")


# Auto-launch Jupyter interface if in a Jupyter environment and script run directly
if IS_COLAB or ('ipykernel' in sys.modules):
    print("📓 Auto-launching Jupyter interface...")
    launch_system('jupyter')

print("✅ Box 3 initialization complete!")
print("🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)")

# Export launch function
__all__ = ['launch_system']


✅ Box 3 initialization complete!
🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)
